# Stock Owl

在台灣投資美股有個最惱人的地方，就是美股開盤都在半夜。如果要時時守著盤勢，對一般人來說還是太吃不消了。為了能夠協助隨時追蹤並且在發生緊急事件時能夠告警，我們列出需要完成的部分：
1. Telegram 機器人
2. Firstrade API 介接

## Telegram 機器人

In [34]:
import telegram
from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler)
from config import financialModelingPrepAPI, TelegramRobotToken

In [35]:
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

In [70]:
CHOOSING, COMFIRM_STOCK, TYPING_CHOICE = range(3)

reply_keyboard = [['輸入追蹤股票', '追蹤標的清單'],
                  ['查詢技術線圖'],
                  ['結束']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)

In [37]:
def start(update, context):
    print(update.effective_chat.id)
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text='''
                             歡迎使用股市貓頭鷹🦉，請選擇你想要執行的服務，我會盡全力協助你的！
                             ''',
                             reply_markup=markup)
    return CHOOSING

In [38]:
def input_stock(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text='''
    請輸入你要追蹤股票代碼：
    （回上一頁請輸入：q）
    ''')
    return TYPING_CHOICE

In [39]:
def catch_stock_info(stock_id):
    import requests
    exchanges = ['NASDAQ', 'NYSE']
    stock_list = []

    for exchange in exchanges:
        res = requests.get('https://financialmodelingprep.com/api/v3/search?query={query}&limit=10&exchange={exchange}&apikey={apikey}'\
                           .format(query = stock_id, exchange = exchange, apikey = financialModelingPrepAPI))
        stock_list += res.json()
    return stock_list

In [130]:
def check_and_store(update, context):
    text = update.message.text
    stock_list = catch_stock_info(text)
    
    if not stock_list:
        context.bot.send_message(chat_id=update.effective_chat.id, text='''
        查詢不到你輸入的股票代碼: {} 哦，要不要檢查一下該股票代碼是否有在美股市場上市呢？再重新輸入一次吧！（回上一頁請輸入：q）
        '''.format(text))
        return TYPING_CHOICE
    print(stock_list)
    stocklist_keyboard = [[stock['symbol']] for stock in stock_list]
    context.user_data['temp_stock_list'] = stock_list
    # [['\r'.join(['{}:{}'.format(k, v)]) for k, v in stock.items()] for stock in stock_list]
    stocklist_markup = ReplyKeyboardMarkup(stocklist_keyboard, one_time_keyboard=True)
    context.bot.send_message(chat_id=update.effective_chat.id,
                              text='''
                              你查詢的結果如下，請問你想加入的是第幾支標的呢？\r\n（回上一頁請輸入：q）
                              ''',
                              reply_markup=stocklist_markup
                             )
    return COMFIRM_STOCK

In [131]:
def stock_added(update, context):
    text = update.message.text
    print(context.user_data)
    if text not in context.user_data['temp_stock_list']:
        stocklist_keyboard = [[stock['symbol']] for stock in context.user_data['temp_stock_list']]
        context.bot.send_message(chat_id=update.effective_chat.id, 
                             text='''
                             你輸入的標的不在清單中哦！請重新輸入\r\n（回上一頁請輸入：q）
                             '''.format(text),
                             reply_markup=stocklist_keyboard)
        return COMFIRM_STOCK
        
    if 'stock_list' in context.user_data:
        context.user_data['stock_list'].append(text)
    else:
        context.user_data['stock_list'] = [text]
    
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text='''
                             已經成功加入 {} 到追蹤清單囉！\r\n請選擇你想查詢的服務🦉
                             '''.format(text),
                             reply_markup=markup)
    print(context.user_data['stock_list'])
    return CHOOSING

In [132]:
def done(update, context):
    user_data = context.user_data
    if 'stock_list' in user_data:
        del user_data['stock_list']

    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text = "再見囉！希望有幫助到你！還需要我的話，請輸入 /start 我就會出現囉🦉")

    user_data.clear()
    return ConversationHandler.END

In [133]:
def get_k_plot(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text = "還沒建置完哦，抱歉！請選擇其他功能吧！",
                             reply_markup=markup)
    return CHOOSING

In [134]:
def check_stock_list(update, context):
    user_data = context.user_data
    if 'stock_list' not in user_data:
        context.bot.send_message(chat_id=update.effective_chat.id, 
                                 text='''
                                 你還沒有加入追蹤的股票哦！請點選『輸入追蹤股票』來追蹤你感興趣的標的吧！
                                 '''.format(text),
                                 reply_markup=markup)
        return CHOOSING
    
    stocklist_keyboard = user_data['stock_list']
    context.bot.send_message(chat_id=update.effective_chat.id, 
                                 text='''
                                 你目前追蹤的標的如下：\r\n{}
                                 '''.format('\r\n'.join(stocklist_keyboard)),
                                 reply_markup=markup)
    return CHOOSING

In [135]:
def wrong_command(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text='''
                             不好意思，我不懂你的意思。你可以點選對話框右手邊的選單來與我互動哦！
                             ''',
                             reply_markup=markup)
    return CHOOSING

In [136]:
def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater(TelegramRobotToken, use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CHOOSING: [MessageHandler(Filters.regex('^輸入追蹤股票$') & ~(Filters.command | Filters.regex('^結束$')),
                                      input_stock),
                       MessageHandler(Filters.regex('^查詢技術線圖$') & ~(Filters.command | Filters.regex('^結束$')),
                                      get_k_plot),
                       MessageHandler(Filters.regex('^追蹤標的清單$') & ~(Filters.command | Filters.regex('^結束$')),
                                      check_stock_list),
                       MessageHandler(~(Filters.command | Filters.regex('^結束$')),
                                      wrong_command)
                       ],

            TYPING_CHOICE: [
                MessageHandler(Filters.text & ~(Filters.command | Filters.regex('^q$')),
                               check_and_store),
                MessageHandler(Filters.regex('^q$') & ~(Filters.command),
                               start)],

            COMFIRM_STOCK: [
                MessageHandler(Filters.text & ~(Filters.command | Filters.regex('^q$')),
                               stock_added),
                MessageHandler(Filters.regex('^q$') & ~(Filters.command),
                               start)],
        },

        fallbacks=[MessageHandler(Filters.regex('^結束$'), done)]
    )

    dp.add_handler(conv_handler)
    
    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [137]:
if __name__ == '__main__':
    main()

1257751303
[{'symbol': 'JD', 'name': 'JD.com, Inc.', 'currency': 'USD', 'stockExchange': 'NasdaqGS', 'exchangeShortName': 'NASDAQ'}, {'symbol': 'CJJD', 'name': 'China Jo-Jo Drugstores, Inc.', 'currency': 'USD', 'stockExchange': 'NasdaqCM', 'exchangeShortName': 'NASDAQ'}, {'symbol': 'JDD', 'name': 'Nuveen Diversified Dividend and Income Fund', 'currency': 'USD', 'stockExchange': 'NYSE', 'exchangeShortName': 'NYSE'}, {'symbol': 'DJD', 'name': 'Invesco Dow Jones Industrial Average Dividend ETF', 'currency': 'USD', 'stockExchange': 'NYSEArca', 'exchangeShortName': 'NYSE'}, {'symbol': 'JDIV', 'name': 'JPMorgan U.S. Dividend ETF', 'currency': 'USD', 'stockExchange': 'NYSEArca', 'exchangeShortName': 'NYSE'}, {'symbol': 'JDST', 'name': 'Direxion Daily Junior Gold Miners Index Bear 2X Shares', 'currency': 'USD', 'stockExchange': 'NYSEArca', 'exchangeShortName': 'NYSE'}, {'symbol': 'SOJD', 'name': 'Southern Company (The) Series 2', 'currency': 'USD', 'stockExchange': 'NYSE', 'exchangeShortName':

2020-09-09 00:33:15,567 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...


In [67]:
updater.stop()